In [1]:
from geneformer import Classifier

import numpy as np

/home/syyang/python_virtuenv/geneformer_3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Geneformer_6layer_model_PATH ='/home/syyang/Projects/geneformer/Geneformer'

Tokenized_adipose_PATH = "/home/syyang/GitRepo/_former/geneformer_results/adipose/output_prefix.dataset"



Finetune_classifier_output_PATH =  "/home/syyang/GitRepo/_former/geneformer_results/adipose/finetune_classifier_output"




output_prefix='classifier_finetune_output'

In [3]:
filter_data_dict={"cell_type":["Cardiomyocyte1","Cardiomyocyte2","Cardiomyocyte3"]}
training_args = {
    "num_train_epochs": 0.9,
    "learning_rate": 0.000804,
    "lr_scheduler_type": "polynomial",
    "warmup_steps": 1812,
    "weight_decay":0.258828,
    "per_device_train_batch_size": 12,
    "seed": 73,
}
cc = Classifier(classifier="cell",
                cell_state_dict = {"state_key": "cell_type", "states": "all"},
                training_args=training_args,
                max_ncells=None,
                freeze_layers = 2,
                num_crossval_splits = 1,
                forward_batch_size=200,
                nproc=16)

In [19]:
## Known there are 71200 cells in total 

np.random.seed(73)
train_ids, eval_ids, test_ids = np.split(np.random.permutation(71200), [int(71200*0.7), int(71200*0.85)])
train_ids = [str(i) for i in train_ids]
eval_ids = [str(i) for i in eval_ids]
test_ids = [str(i) for i in test_ids]

train_test_id_split_dict = {"attr_key": "cell_type",
                            "train": train_ids+eval_ids,
                            "test": test_ids}




In [22]:
cc.prepare_data(input_data_file=Tokenized_adipose_PATH,
                output_directory=Finetune_classifier_output_PATH,
                output_prefix=output_prefix)
                #split_id_dict=train_test_id_split_dict)

Saving the dataset (1/1 shards): 100%|██████████| 7120/7120 [00:00<00:00, 198362.30 examples/s]


In [4]:
# run in the terminal before re-run the following code
# rm -rf  ~/GitRepo/_former/geneformer_results/adipose/finetune_classifier_output/240512_geneformer_cellClassifier_classifier_finetune_output/

# 6 layer Geneformer: https://huggingface.co/ctheodoris/Geneformer/blob/main/model.safetensors
all_metrics = cc.validate(model_directory=f"{Geneformer_6layer_model_PATH}",
                          prepared_input_data_file=f"{Finetune_classifier_output_PATH}/{output_prefix}_labeled_train.dataset",
                          id_class_dict_file=f"{Finetune_classifier_output_PATH}/{output_prefix}_id_class_dict.pkl",
                          output_directory=Finetune_classifier_output_PATH,
                          output_prefix=output_prefix)

mkdir: cannot create directory ‘/home/syyang/GitRepo/_former/geneformer_results/adipose/finetune_classifier_output/240512_geneformer_cellClassifier_classifier_finetune_output/’: File exists
  0%|          | 0/1 [00:00<?, ?it/s]

mkdir: cannot create directory ‘/home/syyang/GitRepo/_former/geneformer_results/adipose/finetune_classifier_output/240512_geneformer_cellClassifier_classifier_finetune_output/ksplit1’: File exists
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/syyang/Projects/geneformer/Geneformer and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/syyang/python_virtuenv/geneformer_3.11/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(d

****** Validation split: 1/1 ******



/home/syyang/python_virtuenv/geneformer_3.11/lib/python3.11/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
0,0.487600,0.425399,0.866732,0.809458


  0%|          | 0/1 [13:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.50 GiB. GPU 0 has a total capacity of 23.65 GiB of which 7.22 GiB is free. Including non-PyTorch memory, this process has 16.24 GiB memory in use. Of the allocated memory 14.19 GiB is allocated by PyTorch, and 1.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 